In [27]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('data/cardekho_used_cars.csv')

In [67]:
df_copy = df.copy()

In [68]:
df_copy.drop('url',axis = 1,inplace=  True)

### Data cleaning and EDA

In [71]:
#removing year from title column

df_copy['name'] = df_copy['title'].str[5:]
df_copy.head()

,name,title,price,registration_yr,insurance,fuel_type,seats,km_driven,ownership,engine_displacement,transmission,manufacture_yr
0,Tata Nexon Creative Plus S,2024 Tata Nexon Creative Plus S,₹9.97 Lakh,2024,-,Petrol,5 Seats,"10,000 Kms",First Owner,1199 cc,Manual,2024
1,Kia Sonet HTX Plus Turbo iMT DT,2022 Kia Sonet HTX Plus Turbo iMT DT,₹8.50 LakhMake Your Offer,Feb 2022,Comprehensive,Petrol,5 Seats,"28,000 Kms",First Owner,998 cc,Manual,2022
2,Tata Hexa XTA,2018 Tata Hexa XTA,₹7.25 LakhMake Your Offer,Mar 2018,Comprehensive,Diesel,7 Seats,"82,000 Kms",First Owner,2179 cc,Automatic,2018
3,Kia Seltos HTX IVT G,2020 Kia Seltos HTX IVT G,₹9.50 Lakh,2020,Zero Dep,Petrol,5 Seats,"90,000 Kms",First Owner,1497 cc,Automatic,2020
4,Hyundai Creta 1.4 E Plus,2019 Hyundai Creta 1.4 E Plus,₹7.35 LakhMake Your Offer,May 2019,-,Diesel,5 Seats,"70,000 Kms",First Owner,1396 cc,Manual,2019


In [72]:
#bringing the name columns forward

cols = df_copy.columns.tolist()
cols = ['name'] + [col for col in cols if col != 'name']
df_copy = df_copy[cols]

df_copy.drop('title',axis = 1, inplace=True)                        #dropping title column since we don't need it anymore

In [73]:
#seprating brand name from name column

df_copy['brand'] = df_copy['name'].str.split().str[0]


In [74]:
#bringing the brand columns forward

cols = df_copy.columns.tolist()
cols = ['brand'] + [col for col in cols if col != 'brand']
df_copy = df_copy[cols]

In [75]:
df_copy['price'] = df_copy['price'].str[1:]
df_copy['price'] = df_copy['price'].str.replace('Make Your Offer','')
df_copy['price'] = df_copy['price'].str[:-4]
df_copy['price'] = df_copy['price'].astype(float)
df_copy['price'] = round(df_copy['price'] * 100000,1)

In [76]:
#registration year and manufacture year is same for all the cars with few neglible rows 
#but just for learning purpose i went ahead with cleaning registration year column

df_copy['registration_yr'] = pd.to_datetime(df_copy['registration_yr'], format= 'mixed')   
df_copy['registration_yr'] = df_copy['registration_yr'].dt.year

In [77]:
df_copy['insurance'].value_counts()         #108 of total rows are unknown so i'll just fill it as 'Own Damage' since it's one of the most common car insurance type in India

insurance
Comprehensive    199
-                193
Zero Dep          37
Third Party       22
Name: count, dtype: int64

In [78]:
df_copy['insurance'] = df_copy['insurance'].replace('-','Own Damage')
print(df_copy['insurance'].value_counts())

insurance
Comprehensive    199
Own Damage       193
Zero Dep          37
Third Party       22
Name: count, dtype: int64


In [79]:
df_copy['seats'] = df_copy['seats'].str[:1]


In [80]:
df_copy['km_driven'] = df_copy['km_driven'].str[:-3]
df_copy['km_driven'] = df_copy['km_driven'].str.replace(',','')

In [81]:
df_copy['engine_displacement'] = df_copy['engine_displacement'].str.replace('cc','')

In [95]:
df_copy.drop(columns=['title'],inplace=True)

#### save point

In [ ]:
df_copy.to_csv('data/cleaned_cars_data.csv',index = False)

In [28]:
final_df = pd.read_csv('data/cleaned_cars_data.csv')

In [29]:
final_df.head()

,brand,name,price,registration_yr,insurance,fuel_type,seats,km_driven,ownership,engine_displacement,transmission,manufacture_yr
0,Tata,Tata Nexon Creative Plus S,997000.0,2024,Own Damage,Petrol,5,10000,First Owner,1199.0,Manual,2024
1,Kia,Kia Sonet HTX Plus Turbo iMT DT,850000.0,2022,Comprehensive,Petrol,5,28000,First Owner,998.0,Manual,2022
2,Tata,Tata Hexa XTA,725000.0,2018,Comprehensive,Diesel,7,82000,First Owner,2179.0,Automatic,2018
3,Kia,Kia Seltos HTX IVT G,950000.0,2020,Zero Dep,Petrol,5,90000,First Owner,1497.0,Automatic,2020
4,Hyundai,Hyundai Creta 1.4 E Plus,735000.0,2019,Own Damage,Diesel,5,70000,First Owner,1396.0,Manual,2019


In [30]:
print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                451 non-null    object 
 1   name                 451 non-null    object 
 2   price                451 non-null    float64
 3   registration_yr      451 non-null    int64  
 4   insurance            451 non-null    object 
 5   fuel_type            451 non-null    object 
 6   seats                451 non-null    int64  
 7   km_driven            451 non-null    int64  
 8   ownership            448 non-null    object 
 9   engine_displacement  448 non-null    float64
 10  transmission         451 non-null    object 
 11  manufacture_yr       451 non-null    int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 42.4+ KB
None


In [31]:
final_df.dropna(ignore_index=True, inplace=True)

In [32]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                445 non-null    object 
 1   name                 445 non-null    object 
 2   price                445 non-null    float64
 3   registration_yr      445 non-null    int64  
 4   insurance            445 non-null    object 
 5   fuel_type            445 non-null    object 
 6   seats                445 non-null    int64  
 7   km_driven            445 non-null    int64  
 8   ownership            445 non-null    object 
 9   engine_displacement  445 non-null    float64
 10  transmission         445 non-null    object 
 11  manufacture_yr       445 non-null    int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 41.8+ KB


In [33]:
final_df.head()

,brand,name,price,registration_yr,insurance,fuel_type,seats,km_driven,ownership,engine_displacement,transmission,manufacture_yr
0,Tata,Tata Nexon Creative Plus S,997000.0,2024,Own Damage,Petrol,5,10000,First Owner,1199.0,Manual,2024
1,Kia,Kia Sonet HTX Plus Turbo iMT DT,850000.0,2022,Comprehensive,Petrol,5,28000,First Owner,998.0,Manual,2022
2,Tata,Tata Hexa XTA,725000.0,2018,Comprehensive,Diesel,7,82000,First Owner,2179.0,Automatic,2018
3,Kia,Kia Seltos HTX IVT G,950000.0,2020,Zero Dep,Petrol,5,90000,First Owner,1497.0,Automatic,2020
4,Hyundai,Hyundai Creta 1.4 E Plus,735000.0,2019,Own Damage,Diesel,5,70000,First Owner,1396.0,Manual,2019


Splitting before feature engineering to avoid data leakage and overfitting

In [34]:
X = final_df.drop(columns=['name','price','registration_yr'])
y = final_df[['price']]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .2, random_state = 42)

In [36]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [37]:
X_train

,brand,insurance,fuel_type,seats,km_driven,ownership,engine_displacement,transmission,manufacture_yr
0,Toyota,Own Damage,Petrol,5,80000,First Owner,1462.0,Manual,2022
1,Hyundai,Zero Dep,Diesel,5,70000,First Owner,1582.0,Manual,2018
2,Toyota,Own Damage,Petrol,5,77000,First Owner,1496.0,Automatic,2018
3,Kia,Comprehensive,Petrol,7,38000,First Owner,1353.0,Automatic,2022
4,Mahindra,Own Damage,Diesel,7,22000,First Owner,2198.0,Automatic,2024
...,...,...,...,...,...,...,...,...,...
351,Maruti,Comprehensive,Petrol,5,46000,First Owner,1197.0,Manual,2018
352,Audi,Comprehensive,Petrol,5,45068,First Owner,1395.0,Automatic,2016
353,Hyundai,Zero Dep,Diesel,5,83000,First Owner,1582.0,Automatic,2020
354,Toyota,Comprehensive,CNG,5,27000,First Owner,1197.0,Manual,2024


In [38]:
num_features = ['seats','km_driven','engine_displacement','manufacture_yr']

num_pipeline = Pipeline(steps = [
                        ('imputer', SimpleImputer(strategy= 'median')),
                        ('scaler',StandardScaler())
                        ])

In [39]:
cat_features = ['insurance','fuel_type','ownership','transmission']

cat_pipeline = Pipeline(steps=[
                        ('imputer',SimpleImputer(strategy='most_frequent')),
                        ('onehotenc',OneHotEncoder(handle_unknown='ignore'))
])

In [40]:
target_enc_feature = ['brand']

target_enc_pipeline = Pipeline(steps=[
                                ('imputer',SimpleImputer(strategy='most_frequent')),
                                ('target_enc',TargetEncoder(smoothing=10))
])

In [41]:
preprocesser = ColumnTransformer(transformers=[
                                                ('num',num_pipeline, num_features),
                                                ('cat',cat_pipeline,cat_features),
                                                ('target_enc',target_enc_pipeline,target_enc_feature)
])

In [42]:
preprocesser

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e

#### Ridge model

In [43]:
model = Pipeline(steps=[
                        ('preprocesser',preprocesser),
                        ('regressor',Ridge(alpha=1))
])

In [44]:
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

In [45]:
mae = mean_absolute_error(y_test, y_pred)
print(f'mae: {mae}')
mse = mean_squared_error(y_test,y_pred)
print(f'mse: {mse}')
rmse = np.sqrt(mse)
print(f'rmse: {rmse}')
r2 = r2_score(y_test, y_pred)
print(f'r2 score: {r2}')

mae: 210825.58428926318
mse: 80126878068.34596
rmse: 283066.91447137715
r2 score: 0.6334720264498792


In [46]:
results_df = X_test.copy()
results_df['y_test'] = y_test
results_df['y_pred'] = y_pred
results_df['error'] = results_df['y_pred'] - results_df['y_test']

In [47]:
results_df.head(10)

,brand,insurance,fuel_type,seats,km_driven,ownership,engine_displacement,transmission,manufacture_yr,y_test,y_pred,error
0,Kia,Zero Dep,Petrol,6,24000,First Owner,1482.0,Automatic,2023,1525000.0,1.466060e+06,-58940.358361
1,Maruti,Zero Dep,Petrol,4,19000,First Owner,1462.0,Manual,2023,1175000.0,8.603770e+05,-314622.984842
2,Hyundai,Comprehensive,Petrol,5,54000,Second Owner,1197.0,Manual,2012,140000.0,1.435369e+05,3536.908094
3,Tata,Own Damage,Petrol,5,31000,First Owner,1198.0,Manual,2019,565000.0,6.068615e+05,41861.501626
4,Tata,Own Damage,Petrol,5,39000,First Owner,1199.0,Manual,2022,685000.0,7.929797e+05,107979.712667
5,Maruti,Comprehensive,Petrol,6,56000,First Owner,1462.0,Manual,2021,825000.0,8.258889e+05,888.905941
6,Skoda,Comprehensive,Petrol,5,24000,First Owner,999.0,Automatic,2022,1096000.0,1.139018e+06,43018.026604
7,Maruti,Comprehensive,Petrol,6,21000,First Owner,1462.0,Manual,2021,950000.0,8.723341e+05,-77665.895044
8,Mahindra,Own Damage,Diesel,4,30000,First Owner,2184.0,Manual,2022,1350000.0,1.370433e+06,20433.080612
9,Hyundai,Own Damage,Petrol,5,39000,First Owner,1497.0,Manual,2023,1295000.0,9.548648e+05,-340135.222828


In [48]:
ohe = model.named_steps['preprocesser'] \
          .named_transformers_['cat'] \
          .named_steps['onehotenc']

ohe_feature_names = ohe.get_feature_names_out()
print(ohe_feature_names)

['x0_Comprehensive' 'x0_Own Damage' 'x0_Third Party' 'x0_Zero Dep'
 'x1_CNG' 'x1_Diesel' 'x1_Petrol' 'x2_First Owner' 'x2_Second Owner'
 'x2_Third Owner' 'x3_Automatic' 'x3_Manual']


In [49]:
X_train_enc = model.named_steps['preprocesser'].transform(X_train)

encoded_df = pd.DataFrame(
    X_train_enc,
    columns=(
        num_features +
        list(ohe_feature_names) +
        ['brand_target_encoded']
    )
)

encoded_df.head()


,seats,km_driven,engine_displacement,manufacture_yr,x0_Comprehensive,x0_Own Damage,x0_Third Party,x0_Zero Dep,x1_CNG,x1_Diesel,x1_Petrol,x2_First Owner,x2_Second Owner,x2_Third Owner,x3_Automatic,x3_Manual,brand_target_encoded
0,-0.421148,1.016111,-0.212091,0.515955,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.255049e+06
1,-0.421148,0.700623,0.116785,-0.844638,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,9.026260e+05
2,-0.421148,0.921465,-0.118909,-0.844638,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.255049e+06
3,2.057012,-0.308938,-0.510819,0.515955,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.249117e+06
4,2.057012,-0.813718,1.805012,1.196252,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.370005e+06


#### Random Forest Regressor

In [50]:
rf_model = Pipeline([
                ('preprocesser', preprocesser),
                ('regressor',RandomForestRegressor(n_estimators= 300,
                                                   random_state = 42))
])

In [51]:
rf_model.fit(X_train,y_train)
y_rf_pred = rf_model.predict(X_test)

d:\VS_adv_python\house_pricing\.hpvenv\Lib\site-packages\sklearn\base.py:1336: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [52]:
mae = mean_absolute_error(y_test, y_rf_pred)
print(f'mae: {mae}')
mse = mean_squared_error(y_test,y_rf_pred)
print(f'mse: {mse}')
rmse = np.sqrt(mse)
print(f'rmse: {rmse}')
r2 = r2_score(y_test, y_rf_pred)
print(f'r2 score: {r2}')

mae: 155723.42027822364
mse: 54714070727.98017
rmse: 233910.3903805476
r2 score: 0.7497189713107397


we have some improvements, the errors have decreased overall in the model.

#### CatBoost

as the final model, i'm going with CatBoost since it is one of the gold standard for pricing problems.

- Handles categorical features (like brand) natively
- No need for one-hot or target encoding
- Very strong on small–medium datasets

In [53]:
from catboost import CatBoostRegressor

In [54]:
cat_features = ['brand','insurance','fuel_type','ownership','transmission']

In [55]:
catb_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth = 6,
    loss_function='MAE',
    eval_metric='MAE',
    random_seed=42,
    verbose=100
)

In [56]:
catb_model.fit(X_train,
               y_train,
               cat_features=cat_features,
               eval_set=(X_test,y_test),
               use_best_model=True)

0:	learn: 397237.9211080	test: 389271.9099224	best: 389271.9099224 (0)	total: 166ms	remaining: 2m 45s
100:	learn: 122559.0263474	test: 182651.0327225	best: 182651.0327225 (100)	total: 4.11s	remaining: 36.6s
200:	learn: 80456.7539526	test: 166365.8060242	best: 166217.6617517 (187)	total: 8.33s	remaining: 33.1s
300:	learn: 63071.7228705	test: 161434.6361110	best: 161366.7581784 (299)	total: 11.6s	remaining: 26.9s
400:	learn: 55138.1061665	test: 159248.1342587	best: 159239.8492947 (398)	total: 14.7s	remaining: 21.9s
500:	learn: 49653.7321283	test: 157677.0647245	best: 157594.6229890 (496)	total: 17.7s	remaining: 17.7s
600:	learn: 44885.0528454	test: 157274.6510497	best: 157271.3582033 (598)	total: 20.9s	remaining: 13.9s
700:	learn: 40816.3787843	test: 156484.4399833	best: 156460.1043733 (685)	total: 24s	remaining: 10.2s
800:	learn: 38075.9777978	test: 156623.8639838	best: 156460.1043733 (685)	total: 27s	remaining: 6.7s
900:	learn: 36006.0920052	test: 155446.6616990	best: 155412.6239838 (8

In [57]:
catb_y_pred = catb_model.predict(X_test)

print('MAE:',mean_absolute_error(y_test,catb_y_pred))
print('R2:', r2_score(y_test,catb_y_pred))

MAE: 155412.62398483287
R2: 0.7598231785771014


In [58]:
# feature importance in catboost model

feat_imp = pd.DataFrame({
    'feature': catb_model.feature_names_,
    'importance': catb_model.get_feature_importance()
}).sort_values(by='importance', ascending=False)

feat_imp

,feature,importance
8,manufacture_yr,21.967002
6,engine_displacement,21.475228
7,transmission,16.509418
0,brand,12.845027
4,km_driven,7.774834
2,fuel_type,6.171577
1,insurance,5.768911
5,ownership,3.883536
3,seats,3.604469


scaling features using log to see if the model performance improves

In [59]:
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

In [60]:
catb_model.fit(
    X_train,
    y_train_log,
    cat_features=cat_features,
    eval_set=(X_test, y_test_log),
    use_best_model=True)

0:	learn: 0.4055056	test: 0.4028219	best: 0.4028219 (0)	total: 41ms	remaining: 40.9s
100:	learn: 0.1172957	test: 0.1777554	best: 0.1777554 (100)	total: 3.08s	remaining: 27.4s
200:	learn: 0.0775089	test: 0.1666497	best: 0.1662891 (188)	total: 6.24s	remaining: 24.8s
300:	learn: 0.0588408	test: 0.1620525	best: 0.1619749 (265)	total: 9.39s	remaining: 21.8s
400:	learn: 0.0514034	test: 0.1609615	best: 0.1602778 (367)	total: 12.5s	remaining: 18.6s
500:	learn: 0.0448265	test: 0.1594532	best: 0.1594532 (500)	total: 15.5s	remaining: 15.4s
600:	learn: 0.0398603	test: 0.1578706	best: 0.1578706 (600)	total: 18.5s	remaining: 12.3s
700:	learn: 0.0361652	test: 0.1571105	best: 0.1569202 (667)	total: 21.6s	remaining: 9.23s
800:	learn: 0.0338046	test: 0.1561611	best: 0.1561151 (796)	total: 24.7s	remaining: 6.13s
900:	learn: 0.0313383	test: 0.1553308	best: 0.1553308 (900)	total: 27.7s	remaining: 3.04s
999:	learn: 0.0293921	test: 0.1548533	best: 0.1548533 (999)	total: 31.2s	remaining: 0us

bestTest = 0.154

In [61]:
catb_y_pred_log = catb_model.predict(X_test)
y_pred = np.exp(catb_y_pred_log)
y_true = np.exp(y_test_log)

In [70]:
catboost_feature_importance = catb_model.get_feature_importance()

importance_df = pd.DataFrame({'Feature': X_train.columns,
                              'Importance': catboost_feature_importance})

importance_df = importance_df.sort_values(by = 'Importance', ascending = False)

print(importance_df)

               Feature  Importance
8       manufacture_yr   28.489699
6  engine_displacement   19.919204
7         transmission   14.597488
0                brand   13.271813
4            km_driven    7.055902
2            fuel_type    5.288382
1            insurance    4.954696
5            ownership    4.523170
3                seats    1.899648


In [62]:
print('MAE:',mean_absolute_error(y_true,y_pred))
print('R2:', r2_score(y_true,y_pred))

MAE: 146008.55613280425
R2: 0.792921148896046


In [ ]:
results = X_test.copy()
results['y_test'] = y_test.values
results['y_cb_pred'] = y_pred
results['error'] = results['y_cb_pred'] - results['y_test']

In [68]:
results.head()

,brand,insurance,fuel_type,seats,km_driven,ownership,engine_displacement,transmission,manufacture_yr,y_test,y_cb_pred,error
0,Kia,Zero Dep,Petrol,6,24000,First Owner,1482.0,Automatic,2023,1525000.0,1.561305e+06,36305.198170
1,Maruti,Zero Dep,Petrol,4,19000,First Owner,1462.0,Manual,2023,1175000.0,1.113232e+06,-61768.184494
2,Hyundai,Comprehensive,Petrol,5,54000,Second Owner,1197.0,Manual,2012,140000.0,1.849797e+05,44979.737257
3,Tata,Own Damage,Petrol,5,31000,First Owner,1198.0,Manual,2019,565000.0,4.446160e+05,-120384.021961
4,Tata,Own Damage,Petrol,5,39000,First Owner,1199.0,Manual,2022,685000.0,7.267652e+05,41765.170433


In [69]:
results.to_csv('results/catboost_results.csv')

In [71]:
results.describe()

,seats,km_driven,engine_displacement,manufacture_yr,y_test,y_cb_pred,error
count,89.000000,89.000000,89.000000,89.000000,8.900000e+01,8.900000e+01,8.900000e+01
mean,5.325843,41609.640449,1476.314607,2020.640449,1.036449e+06,1.053538e+06,1.708841e+04
std,0.765377,24038.683132,405.798976,3.148763,4.702071e+05,4.244031e+05,2.132808e+05
min,4.000000,5700.000000,998.000000,2011.000000,1.400000e+05,1.849797e+05,-4.340851e+05
25%,5.000000,24000.000000,1197.000000,2019.000000,6.990000e+05,7.810923e+05,-9.419318e+04
50%,5.000000,37000.000000,1462.000000,2022.000000,9.500000e+05,1.034111e+06,6.092268e+03
75%,5.000000,60000.000000,1497.000000,2023.000000,1.350000e+06,1.304392e+06,9.061888e+04
max,8.000000,112000.000000,2773.000000,2025.000000,2.000000e+06,2.017187e+06,1.029654e+06


In [64]:
catb_model.save_model("models/catboost_model.cbm")